# 01_02_move_fits_files_using_keyword

## 필요한 모듈

이 프로젝트를 위해서는 아래의 모듈이 필요하다. 

> numpy, pandas, matplotlib, astropy, version_information

### 모듈 설치

1. 콘솔 창에서 모듈을 설치할 때는 아래와 같은 형식으로 입력하면 된다.

>pip install module_name==version

>conda install module_name=version

2. 주피터 노트북(코랩 포함)에 설치 할 때는 아래의 셀을 실행해서 실행되지 않은 모듈을 설치할 수 있다. (pip 기준) 만약 아나콘다 환경을 사용한다면 7행을 콘다 설치 명령어에 맞게 수정하면 된다.

In [ ]:
#import sys
#!pip install astropy==5.2 photutils==1.6 #astroscrappy==1.1.1
#%pip install astroscrappy==1.1.1

In [ ]:
import importlib, sys, subprocess
packages = "numpy, pandas, matplotlib, scipy, astropy, photutils, ccdproc, astroscrappy, version_information" # required modules
pkgs = packages.split(", ")
for pkg in pkgs :
    if not importlib.util.find_spec(pkg):
        #print(f"**** module {pkg} is not installed")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg, '-q'])
    else: 
        print(f"**** module {pkg} is installed")

%load_ext version_information
import time
now = time.strftime("%Y-%m-%d %H:%M:%S (%Z = GMT%z)")
print(f"This notebook was generated at {now} ")

vv = %version_information {packages}
for i, pkg in enumerate(vv.packages):
    print(f"{i} {pkg[0]:10s} {pkg[1]:s}")

### import modules

In [ ]:
from glob import glob
from pathlib import Path, PosixPath, WindowsPath
import os
from astropy.time import Time
from datetime import datetime, timedelta
from astropy.io import fits
import shutil 

import ysfitsutilpy as yfu
#import ysphotutilpy as ypu
#import ysvisutilpy as yvu

import _astro_utilities
import _Python_utilities

In [ ]:
#######################################################
# read all files in base directory for processing
BASEDIR = Path(r"r:\CCD_obs") 
BASEDIR = Path("/mnt/Rdata/OBS_data") 
#BASEDIR = Path("/mnt/OBS_data") 
DOINGDIR = ( BASEDIR/ _astro_utilities.CCD_NEW_dir)
                
DOINGDIRs = sorted(_Python_utilities.getFullnameListOfallsubDirs(str(DOINGDIR)))
#print ("DOINGDIRs: ", format(DOINGDIRs))
print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))

In [ ]:
fnameKEYs = ["OBJECT", "IMAGETYP", "FILTER", "DATE-OBS", 
            "EXPOSURE", "OPTIC", "CCDNAME", "CCD-TEMP", "XBINNING"]

In [ ]:
#%%
for DOINGDIR in DOINGDIRs[30:31] : 
    DOINGDIR = Path(DOINGDIR)
    
    print(f"Starting: {str(DOINGDIR.parts[-1])}")
    summary = yfu.make_summary(DOINGDIR/"*.fit*",)
    if summary is not None : 
        print("summary: ", summary)
        print("len(summary)", len(summary))

        for _, row in summary.iterrows():
            fpath = Path(row["file"])
            print (f"starting {fpath.name}...")

            new_fname = ""

            for KEY in fnameKEYs :
                if KEY in ["OBJECT", "IMAGETYP", "FILTER", 
                    "OPTIC", "CCDNAME", "CCD-TEMP"] :
                    new_fname += str(row[KEY])
                    new_fname += "_"
                
                if KEY == "DATE-OBS" : 
                    new_fname += row[KEY][:19].replace("T","-").replace(":","-")+"_"

                if KEY == "EXPOSURE" : 
                    new_fname += str(int(row[KEY]))+"sec_"

                if KEY == "CCD-TEMP" : 
                    new_fname += str(int(row[KEY]))+"c_"
                
                if KEY == "XBINNING" : 
                    new_fname += str(row[KEY])+"bin.fit"
                
            print(new_fname)

In [ ]:
for _, row in summary.iterrows():
    fpath = Path(row["file"])
    print (f"starting {fpath.name}...")


In [ ]:
summary[fnameKEYs]

In [ ]:
for _, row in summary.iterrows():
    fpath = Path(row["file"])
    print (f"starting {fpath.name}...")

    new_fname = ""

    for KEY in fnameKEYs :
        if KEY in ["OBJECT", "IMAGETYP", "FILTER", 
            "OPTIC", "CCDNAME", "CCD-TEMP"] :
            new_fname += str(row[KEY])
            new_fname += "_"
        
        if KEY == "DATE-OBS" : 
            new_fname += row[KEY][:19].replace("T","-").replace(":","-")+"_"

        if KEY == "EXPOSURE" : 
            new_fname += str(int(row[KEY]))+"sec_"

        if KEY == "CCD-TEMP" : 
            new_fname += str(int(row[KEY]))+"c_"
        
        if KEY == "XBINNING" : 
            new_fname += str(row[KEY])+"bin.fit"
        
    print(new_fname)

In [ ]:
new_folder = _astro_utilities.get_new_foldername_from_filename(new_fname)
new_fpath =  BASEDIR /_astro_utilities.CCD_obs_raw_dir / new_folder / new_fname
print("new_fpath: ", new_fpath)

In [ ]:
for checkKEY in checkKEYs: 
    print(f"{checkKEY}: ", hdul[0].header[checkKEY])

In [ ]:
print(fpath)
new_fname = hdul[0].header["OBJECT"]+"_"+hdul[0].header["IMAGETYP"]+"_"+hdul[0].header["FILTER"]+"_"
new_fname += hdul[0].header["DATE-OBS"][:19].replace("T","-").replace(":","-")+"_"
new_fname += str(int(hdul[0].header["EXPOSURE"]))+"sec_"
new_fname += hdul[0].header["OPTIC"]+"_"+hdul[0].header["CCDNAME"]+"_"
new_fname += str(int(hdul[0].header["CCD-TEMP"]))+"c_"+str(hdul[0].header["XBINNING"])+"bin.fit"
print("new_fname: ", new_fname)

In [ ]:
new_folder = _astro_utilities.get_new_foldername_from_filename(new_fname)
print("new_folder: ", new_folder)


In [ ]:
#new_folder = Path(_astro_utilities.get_new_foldername_from_filename(new_fname))
new_fpath =  BASEDIR /_astro_utilities.CCD_obs_raw_dir / new_folder / new_fname
print("new_fpath: ", new_fpath)

In [ ]:
new_fpath.parents[0]
new_fpath.parents[0].exists()

if not new_fpath.parents[0].exists():
    os.makedirs(f'{new_fpath.parents[0]}')
    print(f'{new_fpath.parents[0]} is created...')  

In [ ]:
if new_fpath.exists():
    duplicate_fpath = BASEDIR / _astro_utilities.CCD_duplicate_dir / new_fpath.name
    os.rename(f"{str(new_fpath)}", 
                f"{str(duplicate_fpath)}")
    print (f"move {str(new_fpath.name)} to {str(duplicate_fpath)}")

#os.rename(f"{str(fpath)}", 
#            f"{str(new_fpath)}")
shutil.move
print (f"move {str(fpath.name)} to {str(new_fpath)}")

In [ ]:
#%%   
#############################################################################
#Check and delete empty folder....
#############################################################################
for i in range(4):
    DOINGDIR = Path( BASEDIR/ _astro_utilities.CCD_NEW_dir)           
    DOINGDIRs = sorted(_Python_utilities.getFullnameListOfallsubDirs(DOINGDIR))
    #print ("DOINGDIRs: ", format(DOINGDIRs))
    print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))

    for DOINGDIR in DOINGDIRs :    
        if len(os.listdir(str(DOINGDIR))) == 0 :
            shutil.rmtree(f"{str(DOINGDIR)}") # Delete..
            print (f"rmtree {str(DOINGDIR)}")
        else : 
            fpaths = _Python_utilities.getFullnameListOfallFiles(str(DOINGDIR))
            print("fpaths", fpaths)

            for fpath in fpaths[:]:
                print("fpath", fpath)

                if fpath[-4:].lower() in [".txt", "xisf", ".zip", ".png", ".log",
                                            "seal", "tiff", ".axy", "atch", "lved",
                                            "rdls", "xyls", "corr", "xosm", ".ini",
                                            ".wcs", ".csv"] \
                                        and os.path.isfile(fpath):
                    os.remove("{}".format(fpath))
                    print("{} is removed...".format(fpath)) 

In [ ]:
fpath = Path("/mnt/homes/parksparks/Drive/KBox/Github/astro_Python/01_08_solving_fits_using_PICs.py")

In [ ]:
fname = fpath.name

In [ ]:
fpath.suffix
fpath.parent